In [ ]:
import json
import os
import re
from datetime import datetime, timedelta

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
from IPython.display import display, HTML

from transaction_history_processor import PortfolioHistory

In [ ]:
if not os.path.exists('portfolio_history.json'):
    print("portfolio_history.json not found. Generating it now...")
    portfolio_history = PortfolioHistory()
    portfolio_history.process_transaction_history(
        input_file='transaction_history.json',
        save_output=True,
        output_file='portfolio_history.json'
    )
    print("portfolio_history.json has been generated.")

with open('portfolio_history.json', 'r') as f:
    data = json.load(f)
    sectors = data['sectors']
    portfolio_data = data['portfolios']

In [ ]:
def plot_stock_proportions(portfolio_data, date=None, show_values=True, main_threshold=3.0):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return None

    holdings = portfolio_data[date]['holdings']
    labels = list(holdings.keys())
    values = [float(data['market_value']) for data in holdings.values()]

    total = sum(values)
    percentages = [(value / total) * 100 for value in values]

    main_holdings = list(zip(labels, values, percentages))
    main_holdings.sort(key=lambda x: x[2], reverse=True)

    other_holdings = [h for h in main_holdings if h[2] < main_threshold]
    if len(other_holdings) > 1:
        others_value = sum(v for _, v, _ in other_holdings)
        others_percentage = sum(p for _, _, p in other_holdings)
        other_symbols_info = "<br>".join(f"<span style='font-family: monospace;'>{l:<5}\t{
                                         p:>6.2f}%</span>" for l, _, p in other_holdings)
        main_holdings = [h for h in main_holdings if h[2] >=
                         main_threshold] + [("Others", others_value, others_percentage)]

    main_labels, main_values, main_percentages = zip(*main_holdings)

    # Create custom hover text
    hover_text = [f"{label}<br>${value:,.2f}<br>{percentage:.2f}%" for label,
                  value, percentage in zip(main_labels, main_values, main_percentages)]
    if "Others" in main_labels:
        others_index = main_labels.index("Others")
        hover_text[others_index] += f"<br><br>{other_symbols_info}"

    fig = go.Figure(data=[go.Pie(labels=main_labels,
                    values=main_percentages,
                    textinfo='label+percent',
                    hovertext=hover_text,
                    hoverinfo='text')])

    fig.update_layout(
        title=f'Stock Proportions by Value on {date}',
        showlegend=True,
        legend=dict(title="Holdings")
    )

    if show_values:
        fig.update_traces(
            text=[f'${value:,.2f}' for value in main_values], textposition='inside')

    return fig

# Call the function
plot_stock_proportions(portfolio_data, show_values=False)

In [ ]:
# plot_stock_proportions(
#     portfolio_history, date="2023-10-26")

In [ ]:
def plot_sector_proportions(portfolio_data, sectors, date=None):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return None

    holdings = portfolio_data[date]['holdings']
    sector_counts = {}
    for symbol, data in holdings.items():
        sector = sectors.get(symbol, 'ETF')
        if sector not in sector_counts:
            sector_counts[sector] = 0
        sector_counts[sector] += float(data['quantity']) * \
            float(data['market_price'])

    labels = list(sector_counts.keys())
    sizes = list(sector_counts.values())

    fig = go.Figure(
        data=[go.Pie(labels=labels, values=sizes, textinfo='label+percent')])

    fig.update_layout(
        title=f'Sector Proportions on {date}',
        showlegend=True,
        legend=dict(title="Sectors")
    )

    return fig

# Call the function with the loaded data
plot_sector_proportions(portfolio_data, sectors)

In [ ]:
def plot_portfolio_growth(portfolio_data):
    dates = sorted(portfolio_data.keys())

    # Exclude the first 'exclude_days' days
    start_date = pd.to_datetime(dates[0])
    dates = [d for d in dates if pd.to_datetime(d) >= start_date]
    unrealized_gains = [
        float(portfolio_data[d]['summary']['unrealized_gain_loss']) for d in dates]
    daily_gains = [float(portfolio_data[d]['summary']['daily_gain']) for d in dates]
    roi = [float(portfolio_data[d]['summary']['roi']) * 100 for d in dates]
    total_market_value = [
        float(portfolio_data[d]['summary']['total_market_value']) for d in dates]
    cash = [
        float(portfolio_data[d]['summary']['cash']) for d in dates]
    cumulative_return = [
        float(portfolio_data[d]['summary']['cumulative_return']) * 100 for d in dates]

    df_plot = pd.DataFrame({
        'Date': pd.to_datetime(dates),
        'Total Market Value': total_market_value,
        'Cash': cash,
        'Unrealized Gain': unrealized_gains,
        'Daily Gain': daily_gains,
        'Cumulative Return (%)': cumulative_return,
        'ROI (%)': roi,
    })

    # Calculate returns for specific periods
    latest_date = df_plot['Date'].max()
    returns = {
        '5D': df_plot['ROI (%)'].iloc[-1] - df_plot[df_plot['Date'] >= latest_date - pd.Timedelta(days=5)]['ROI (%)'].iloc[0],
        '1M': df_plot['ROI (%)'].iloc[-1] - df_plot[df_plot['Date'] >= latest_date - pd.Timedelta(days=30)]['ROI (%)'].iloc[0],
        '6M': df_plot['ROI (%)'].iloc[-1] - df_plot[df_plot['Date'] >= latest_date - pd.Timedelta(days=180)]['ROI (%)'].iloc[0],
        'YTD': df_plot['ROI (%)'].iloc[-1] - df_plot[df_plot['Date'] >= pd.Timestamp(year=latest_date.year, month=1, day=1)]['ROI (%)'].iloc[0],
        '1Y': df_plot['ROI (%)'].iloc[-1] - df_plot[df_plot['Date'] >= latest_date - pd.Timedelta(days=365)]['ROI (%)'].iloc[0],
        'All': df_plot['ROI (%)'].iloc[-1]
    }

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    sp500_data = fetch_sp500_data(
        df_plot['Date'].min(), df_plot['Date'].max())
    sp500_performance = (
        (sp500_data['Close'] - sp500_data['Close'].iloc[0]) / sp500_data['Close'].iloc[0]) * 100

    fig.add_trace(
        go.Scatter(
            x=sp500_data.index,
            y=sp500_performance.round(2),
            line=dict(color='blue', width=2),
            name='S&P 500',
            visible=True
        ),
        secondary_y=False
    )

    fig.add_trace(
        go.Scatter(
            x=df_plot['Date'],
            y=df_plot['ROI (%)'].round(2),
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0.2)',
            line=dict(color='red', width=2),
            name='ROI',
            hovertemplate='ROI: %{y:.2f}% (Cum. Return: %{customdata:.2f}%)<extra></extra>',
            customdata=df_plot['Cumulative Return (%)'].round(2),
            visible=True
        ),
        secondary_y=False
    )

    fig.add_trace(
        go.Scatter(
            x=df_plot['Date'],
            y=df_plot['Total Market Value'].round(2),
            line=dict(color='green', width=2),
            name='Market Value',
            visible=True,
            hovertemplate='$%{y:,.2f} (Cash: $%{customdata:,.2f})',
            customdata=df_plot['Cash'].round(2),
        ),
        secondary_y=True
    )

    fig.add_trace(
        go.Bar(
            x=df_plot['Date'],
            y=df_plot['Unrealized Gain'].round(2),
            name='Unr. Gain',
            marker_color='gray',
            opacity=0.9,
            hovertemplate='$%{y:,.2f} (Daily: $%{customdata:,.2f})',
            customdata=df_plot['Daily Gain'].round(2),
        ),
        secondary_y=True
    )

    performance_text = (
        f"Portfolio ROI:<br>"
        f"<span style='display: inline-block; width: 70px;'>5D: {
            returns['5D']:.2f}%</span> | "
        f"<span style='display: inline-block; width: 80px;'>1M: {
            returns['1M']:.2f}%</span><br>"
        f"<span style='display: inline-block; width: 70px;'>6M: {
            returns['6M']:.2f}%</span> | "
        f"<span style='display: inline-block; width: 80px;'>YTD: {
            returns['YTD']:.2f}%</span><br>"
        f"<span style='display: inline-block; width: 70px;'>1Y: {
            returns['1Y']:.2f}%</span> | "
        f"<span style='display: inline-block; width: 80px;'>All: {
            returns['All']:.2f}%</span>"
    )

    fig.add_annotation(
        x=1, y=1.05, xref='paper', yref='paper',
        text=performance_text,
        showarrow=False,
        font=dict(size=10, family="monospace"),
        align='left',
        xanchor='right',
        yanchor='bottom',
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='rgba(0,0,0,0.5)',
        borderwidth=1,
        borderpad=4
    )

    fig.update_layout(
        title={
            'text': 'Portfolio Performance & Value vs S&P 500',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title='Date',
        hovermode='x unified',
        showlegend=True,
        xaxis=dict(
            rangeslider=dict(visible=False),
            rangeselector=dict(
                buttons=list([
                    dict(count=5, label="5D", step="day", stepmode="backward"),
                    dict(count=1, label="1M", step="month", stepmode="backward"),
                    dict(count=6, label="6M", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(step="all", label="MAX")
                ])
            )
        ),

        margin=dict(t=120)  # Increase top margin to accommodate the annotation
    )

    fig.update_yaxes(
        title_text="Performance (%)",
        ticksuffix='%',
        secondary_y=False
    )
    fig.update_yaxes(
        title_text="Value ($)",
        tickprefix='$',
        secondary_y=True
    )


    return fig


def fetch_sp500_data(start_date, end_date):
    # Convert end_date to datetime and add one day
    end_date = pd.to_datetime(end_date) + timedelta(days=1)

    # Download S&P 500 data
    sp500 = yf.Ticker("^GSPC")
    sp500_data = sp500.history(start=start_date, end=end_date)

    # Return only the 'Close' prices
    return sp500_data[['Close']]

In [ ]:
plot_portfolio_growth(portfolio_data)

In [ ]:
def process_portfolio_data(portfolio_data, date=None, sort_by='Symbol', ascending=True):
    if date is None:
        date = max(portfolio_data.keys())
    elif date not in portfolio_data:
        raise ValueError(f"No data available for date: {date}")
    data = portfolio_data[date]

    # Process holdings data
    holdings_data = [
        {
            'Symbol': symbol,
            'Quantity': round(float(info['quantity']), 2),
            'Unit Cost': round(float(info['unit_cost']), 2),
            'Market Price': round(float(info['market_price']), 2),
            'Total Cost': round(float(info['total_cost']), 2),
            'Market Value': round(float(info['market_value']), 2),
            'Unrealized G/L': f"${float(info['unrealized_gain_loss']):,.2f}",
            'Daily Gain': f"${float(info['daily_gain']):,.2f} ({float(info['daily_return'])*100:.2f}%)"
        }
        for symbol, info in data['holdings'].items()
    ]

    df = pd.DataFrame(holdings_data)
    df = df.sort_values(by=sort_by, ascending=ascending)

    # Process summary data
    summary = data['summary']

    # Process closed positions data
    closed_positions = data.get('closed_positions', {})
    closed_positions_data = [
        {
            'Symbol': symbol,
            'Quantity': round(float(position['quantity']), 2),
            'Sell Price': round(float(position['sell_price']), 2),
            'Cost Basis': round(float(position['cost_basis']), 2),
            'Realized Gain': f"${float(position['realized_gain']):,.2f}"
        }
        for symbol, positions in closed_positions.items()
        for position in positions
    ]

    df_closed = pd.DataFrame(closed_positions_data)
    df_closed = df_closed.sort_values(by='Symbol', ascending=True)

    return {
        'date': date,
        'holdings': df,
        'summary': summary,
        'closed_positions': df_closed,
        'realized_gains': summary.get('realized_gains', 0),
    }

In [ ]:
def display_portfolio_analysis(processed_data):
    date = processed_data['date']
    df = processed_data['holdings']
    summary = processed_data['summary']
    df_closed = processed_data['closed_positions']

    # Display holdings
    display(HTML(f"<h2>Portfolio Status as of {date}</h2>"))
    display(HTML(df.to_html(index=False, classes='dataframe')))

    # Display summary
    print(f"\nPortfolio Summary:")
    print(f"Cash: ${float(summary.get('cash', 0)):,.2f}")
    print(f"Total Market Value: ${
          float(summary.get('total_market_value', 0)):,.2f}")
    print(
        f"Unrealized Gain/Loss: ${float(summary.get('unrealized_gain_loss', 0)):,.2f}")
    print(f"Daily Gain: ${float(summary.get('daily_gain', 0)):,.2f} ({
          float(summary.get('daily_return', 0))*100:.2f}%)")
    print(f"Total Return: {float(summary.get('total_return', 0))*100:.2f}%")
    print(f"Cumulative Return: {
          float(summary.get('cumulative_return', 0))*100:.2f}%")
    print(f"ROI: {float(summary.get('roi', 0))*100:.2f}%")

    # Display closed positions
    display(HTML("<h2>Closed Positions</h2>"))
    display(HTML(df_closed.to_html(index=False, classes='dataframe')))

    # Display total realized gains
    print(f"\nTotal Realized Gains: ${
          float(processed_data['realized_gains']):,.2f}")

In [ ]:
processed_data = process_portfolio_data(portfolio_data)
display_portfolio_analysis(processed_data)

In [ ]:
# processed_data = process_portfolio_data(portfolio_data, date="2024-04-10")
# display_portfolio_analysis(processed_data)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_asset_allocation_over_time(portfolio_data):
    dates = sorted(portfolio_data.keys())
    assets = set()
    for date in dates:
        assets.update(portfolio_data[date]['holdings'].keys())
    assets = sorted(list(assets))

    data = {asset: [] for asset in assets}
    for date in dates:
        total_value = sum(float(info['market_value'])
                          for info in portfolio_data[date]['holdings'].values())
        for asset in assets:
            if asset in portfolio_data[date]['holdings']:
                value = float(portfolio_data[date]
                              ['holdings'][asset]['market_value'])
                percentage = (value / total_value) * 100
            else:
                percentage = 0
            data[asset].append(percentage)

    fig = go.Figure()
    for asset in assets:
        fig.add_trace(go.Scatter(
            x=dates,
            y=data[asset],
            mode='lines',
            stackgroup='one',
            name=asset
        ))

    fig.update_layout(
        title='Asset Allocation Over Time',
        xaxis_title='Date',
        yaxis_title='Percentage',
        yaxis_tickformat=',.0%',
        legend_title='Assets',
        hovermode='x unified'
    )

    return fig


# Call the function
plot_asset_allocation_over_time(portfolio_data)

In [ ]:
def read_template(template_path):
    with open(template_path, 'r') as file:
        return file.read()


def generate_html_report():
    dates = sorted(portfolio_data.keys())
    latest_date = dates[-1]

    stock_proportions_plot = plot_stock_proportions(
        portfolio_data, latest_date)
    sector_proportions_plot = plot_sector_proportions(
        portfolio_data, sectors, latest_date)
    portfolio_growth_plot = plot_portfolio_growth(portfolio_data)
    asset_allocation_plot = plot_asset_allocation_over_time(
        portfolio_data)  # New plot

    processed_data = process_portfolio_data(portfolio_data, latest_date)

    # Convert DataFrames to HTML strings
    portfolio_table = processed_data['holdings'].to_html(
        index=False, classes='dataframe')
    closed_positions_table = processed_data['closed_positions'].to_html(
        index=False, classes='dataframe')

    # Format portfolio summary
    summary = processed_data['summary']
    format_dict = {
        'date': latest_date,
        'portfolio_table': portfolio_table,
        'closed_positions_table': closed_positions_table,
        'stock_proportions_plot': stock_proportions_plot.to_json(),
        'sector_proportions_plot': sector_proportions_plot.to_json(),
        'portfolio_growth_plot': portfolio_growth_plot.to_json(),
        'asset_allocation_plot': asset_allocation_plot.to_json(),  # New plot
        'date_options': ' '.join(f'<option value="{date}">{date}</option>' for date in dates),
        'cash': f"${float(summary.get('cash', 0)):,.2f}",
        'total_market_value': f"${float(summary.get('total_market_value', 0)):,.2f}",
        'unrealized_gain_loss': f"${float(summary.get('unrealized_gain_loss', 0)):,.2f}",
        'daily_gain': f"${float(summary.get('daily_gain', 0)):,.2f} ({float(summary.get('daily_return', 0))*100:.2f}%)",
        'total_realized_gains': f"${float(summary.get('realized_gains', 0)):,.2f}",
        'total_return': f"{float(summary.get('total_return', 0))*100:.2f}%",
        'cumulative_return': f"{float(summary.get('cumulative_return', 0))*100:.2f}%",
        'roi': f"{float(summary.get('roi', 0))*100:.2f}%"
    }

    # Read the HTML template
    template_path = 'reports/templates/portfolio_report_template.html'
    with open(template_path, 'r') as template_file:
        html_template = template_file.read()

    # Read the CSS content
    css_path = 'reports/templates/portfolio_report_styles.css'
    with open(css_path, 'r') as css_file:
        css_content = css_file.read()

    # Escape curly braces in the CSS content
    css_content_escaped = css_content.replace('{', '{{').replace('}', '}}')

    # Replace the CSS placeholder with the escaped CSS content
    html_template = html_template.replace(
        '/* CSS_PLACEHOLDER */', css_content_escaped)

    # Remove whitespace from placeholders
    html_template = re.sub(r'\{\s*(\w+)\s*\}', r'{\1}', html_template)

    # Generate the HTML content
    html_content = html_template.format(**format_dict)

    # Ensure the 'reports' directory exists
    os.makedirs('reports', exist_ok=True)

    # Generate only the 'latest' version
    today_date = datetime.now().strftime("%Y-%m-%d")
    filename = f'portfolio_report_latest_{today_date}.html'
    with open(os.path.join('reports', filename), 'w') as f:
        f.write(html_content)

    print(f"HTML report generated: reports/{filename}")


generate_html_report()